In [ ]:
import re

In [109]:
def isPPline(line):
    return 'MCKESSON' in line

def extract_NDC(line):
    return re.search('[0-9]{11}',line).group(0)

def extract_pp_info(line):
    ndc = extract_NDC(line)
    parts = line.strip().split(ndc)
    description = parts[0]
    data = parts[1].strip().split()[1:]
    return [description,ndc] + data[:3]

def get_data(line):
    ndc = extract_NDC(line)
    parts = line.strip().split(ndc)
    numbers = parts[1].split()
    return [parts[0],ndc] + numbers

In [110]:
f = open('PO.txt')
qoh_sum = 0

columns = []
PPs = []

for line in f.readlines():
    if isPPline(line):
        PPs.append(extract_pp_info(line))
    else:
        columns.append(get_data(line) + [line])

In [118]:
df_pp = pd.DataFrame(PPs,columns=['description','ndc','reorder_units','pkg_size','reorder_pkg'])
df_pp = df_pp.apply(pd.to_numeric,errors='ignore')
df_all_prod = pd.DataFrame(columns,columns=['description','ndc','gcsn','pkg_size','qoh','qoo','full_line'])
df_all_prod = df_all_prod.apply(pd.to_numeric,errors='ignore')

df_pp = df_pp.merge(df_all_prod[['ndc','gcsn','qoh']],on='ndc',how='left')

gcsn_sum = df_all_prod.groupby('gcsn')['qoh'].sum()
gcsn_sum.name = 'total_qoh'
df_gcsn_sum = gcsn_sum.reset_index()

df_pp = df_pp.merge(df_gcsn_sum,on='gcsn', how='left')

df_pp['new_inv_lvl'] = df_pp.qoh + df_pp.reorder_units

In [119]:
df_pp['ndc'] = df_pp.ndc.apply(lambda ndc: '="{}"'.format(str(ndc).zfill(11)))
df_all_prod['ndc'] = df_all_prod.ndc.apply(lambda ndc: '="{}"'.format(str(ndc).zfill(11)))

In [120]:
df_pp

,description,ndc,reorder_units,pkg_size,reorder_pkg,gcsn,qoh,total_qoh,new_inv_lvl
0,ARTIFICIAL TEARS 1.4 % DROPRUG,"=""00536108494""",75.0,15.0,5.0,7936,60,60,135.0
1,CALCIUM CARBONATE 648 MG TARUG,"=""00536102410""",1000.0,1000.0,1.0,59510,44,44,1044.0
2,CLOBETASOL 0.05% GEL PER,"=""45802092514""",60.0,15.0,4.0,21904,60,60,120.0
3,DOC-Q-LACE 100 MG SOFTGEL QUA,"=""00603015032""",2000.0,1000.0,2.0,3009,53,53,2053.0
4,FENOFIBRATE 134 MG CAPSULE MYL,"=""00378862977""",90.0,90.0,1.0,44305,94,94,184.0
5,FLUTICASONE PROP 50 MCG SPRAPO,"=""60505082901""",64.0,16.0,4.0,18368,256,256,320.0
6,GABAPENTIN 100 MG CAPSULE NOR,"=""16714050302""",1500.0,500.0,3.0,21413,0,0,1500.0
7,HUMALOG 100 UNITS/ML VIAL LIL,"=""00002751001""",20.0,10.0,2.0,27413,140,140,160.0
8,HUMULIN N 100 UNITS/ML VIALLIL,"=""00002831501""",40.0,10.0,4.0,1740,160,170,200.0
9,HYDROXYCHLOROQUINE 200 MG TRAN,"=""63304029605""",500.0,500.0,1.0,9580,0,300,500.0


In [121]:
df_all_prod

,description,ndc,gcsn,pkg_size,qoh,qoo,full_line
0,ARTIFICIAL TEARS 1.4 % DROPRUG,"=""00536108494""",7936,15,60,0,ARTIFICIAL TEARS 1.4 % DROPRUG 00536108494...
1,CALCIUM CARBONATE 648 MG TARUG,"=""00536102410""",59510,1000,44,0,CALCIUM CARBONATE 648 MG TARUG 00536102410...
2,CLOBETASOL 0.05% GEL PER,"=""45802092514""",21904,15,60,0,CLOBETASOL 0.05% GEL PER 45802092514...
3,DOC-Q-LACE 100 MG SOFTGEL QUA,"=""00603015032""",3009,1000,53,0,DOC-Q-LACE 100 MG SOFTGEL QUA 00603015032...
4,FENOFIBRATE 134 MG CAPSULE MYL,"=""00378862977""",44305,90,94,0,FENOFIBRATE 134 MG CAPSULE MYL 00378862977...
5,FLUTICASONE PROP 50 MCG SPRAPO,"=""60505082901""",18368,16,256,0,FLUTICASONE PROP 50 MCG SPRAPO 60505082901...
6,FLUTICASONE PROP 50 MCG SPRHI-,"=""50383070016""",18368,16,0,0,FLUTICASONE PROP 50 MCG SPRHI- 50383070016...
7,GABAPENTIN 100 MG CAPSULE NOR,"=""16714050302""",21413,500,0,0,GABAPENTIN 100 MG CAPSULE NOR 16714050302...
8,GABAPENTIN 100 MG CAPSULE ASC,"=""67877022205""",21413,500,0,0,GABAPENTIN 100 MG CAPSULE ASC 67877022205...
9,GABAPENTIN 100 MG CAPSULE CAM,"=""31722022105""",21413,500,0,0,GABAPENTIN 100 MG CAPSULE CAM 31722022105...


In [122]:
ew = pd.ExcelWriter('po_parsed.xlsx')

In [123]:
df_pp.to_excel(ew,'preferred_products')
df_all_prod.to_excel(ew,'all_products')

In [124]:
ew.close()